In [ ]:
import pandas, os, sys, re, collections, importlib, glob

#sys.path.append('/Users/dfporter/pma/')
#import sameRiver
#import sameRiver.biotypeLookupFileMaker
#from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
import Bio
#import pprint
#from pprint import pprint as pp
import primerMaker
importlib.reload(primerMaker)
from primerMaker import primerMaker

from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

import time
import Bio.SeqUtils  # For Bio.SeqUtils.molecular_weight 

In [ ]:
proteins = ['FUBP1', 'HNRNPCL1', 'RPL5', 'TDRKH', 'EEF1B2', 'PCBP1', 'SF3B1', 'PABPC4L', 'YTHDC2', 'BCLAF1', 
            'KHDRBS2', 'RARS2', 'A1CF', 'DDX50', 'UPF2', 'DCP1B', 'NUFIP1', 'DICER1', 'NOVA1', 'CNOT1', 'RBFOX1', 
            'KPNB1', 'SRSF2', 'CRNKL1', 'RBM39', 'RBM11', 'U2AF1', 'DDX3X', 'EIF1AX', 'RPL10', 'RQCD1', 'CNOT9',
           'MYC', 'CDKN2A', 'BRCA1' , 'OAS1',
           'BARD1']
print(proteins, len(proteins))
wells = pandas.read_excel('Sequencing_schemes_and_results.xlsx', sheet_name='midiprep amounts')
prot_to_well = dict(zip(wells.Protein, wells['96 well DNA plate position']))
prot_to_well['PABPC4L'] = prot_to_well['PABPCL4']
prot_to_well['NOVA1'] = prot_to_well['NOVA']
prot_to_well['TDRKH'] = prot_to_well['TDRKH1']
#print(wells.head())

In [ ]:
df = pandas.read_excel('./RBP_high_freq_mutations.xlsx', sheet_name='short_list_cancer')
#df = pandas.read_excel('Sequencing_schemes_and_results.xlsx', sheet_name='midiprep amounts')
#df['Gene'] = df['Protein']
df = df.loc[:, [x for x in df.columns if 'notes' not in x]]
df['URL'] = df['URL'].astype('str')
df['URL'] = [x.split('/')[-1] for x in df.URL]
print(df.head())

In [ ]:
# BamHI cut uORF Nt FHH (only see His tag in overlap primer.)

left_overlap = 'ACATCATCACCACCATCATGGATCC' # Nt FHH insertion overlap.
#left_overlap = 'GGCATAACGACTAATATTTATG' # Ct FHH insertion overlap (BamHI cut)

#right_overlap = 'CCACACTGGACTAGTGGATC' # <- as in the primer (antisense)
right_overlap = 'GATCCACTAGTCCAGTGTGG' # <- Nt FHH insertion overlap. as in the vector (sense)
#right_overlap = 'GATCCAGATTATAAAGATGA'  # Ct FHH insertion overlap (BamHI cut)

remove_stop = True # Remove stop codon from CDS if present.

print(f"left o/l length: {len(left_overlap)}, right o/l length: {len(right_overlap)}")
from Bio.Data import CodonTable
standard_table = CodonTable.unambiguous_dna_by_id[1]


In [ ]:
#https://www.ncbi.nlm.nih.gov/nuccore/BC040523


def get_dna(_loc, seq_record) -> Bio.Seq:
    seq = seq_record.seq
    dna =  seq[_loc._start:_loc._end]#, IUPAC.ambiguous_dna)
    if _loc._strand != 1:
        dna = dna.reverse_complement()
    return dna  # Seq object

def get_cds_from_genbank_record(seq_record: Bio.SeqRecord.SeqRecord) -> tuple:

        # Did we annotate what to PCR?
        highlighted_cds = 0
        starts_with = 0
        for feature in seq_record.features:
            y = feature.__dict__
            
            if 'qualifiers' in y and 'label' in y['qualifiers']:
                
                if y['qualifiers']['label'][0] == 'CDS_to_amplify':
                    highlighted_cds = feature
                    # 0-based.
                    if 'starts_with_aa_num' in y['qualifiers']:
                        starts_with = int(y['qualifiers']['starts_with_aa_num'][0])
                    else:
                        starts_with = 0
        if highlighted_cds:
            cds = highlighted_cds
            
        # If not, get the first CDS annotated.
        else:
            try:
                cds = [x for x in seq_record.features if x.type=='CDS'][0]
            except:
                print("*" * 14 + "No CDS...")
                return
        
        cds_start = cds.location.start
        cds_dna = seq_record.seq[cds.location.start:cds.location.end]
        if cds.location.strand != 1:
            cds_dna = Seq.reverse_complement(cds_dna)
        cds_aa = cds_dna.translate()
        
        # 'RHKDESTNQCUGPAVILMFYW'
        #if not str(cds_aa)[-1].isalpha():
        #    cds_aa = cds_aa[:-1]
            
        return cds_dna, cds_aa, starts_with
    
def get_primers(
    fname: str, aa_wt: str, aa_n: int, aa_mut: str, gene_name: str ='gene_name',
    remove_stop: bool = False) -> str: 
    
    for seq_record in SeqIO.parse(fname, "genbank"):

        print(seq_record.id)
        #print(seq_record.features)

        # cds_aa: Amino acid sequence.
        cds_dna, cds_aa, starts_with = get_cds_from_genbank_record(seq_record)
        
        if remove_stop and (cds_dna[-3:] in ['TAA', 'TAG', 'TGA']):
            #print(f"Converted {cds_dna} to {cds_dna[:-3]}")
            cds_dna = cds_dna[:-3]
            
        if remove_stop and (cds_aa[-1] == '*'):
            cds_aa = cds_aa[:-1]
            
        aa_n -= starts_with
    
        #cds_dna = get_dna(cds.location, seq_record)
        #if 'translation' in cds.qualifiers:
        #    cds_aa = cds.qualifiers['translation'][0]
        #else:
        #    cds_aa = cds.translate(cds_dna)
        print('aa_n = ', aa_n, ' seq length for cds: ', len(cds_dna))
        if aa_n * 3 > len(cds_dna):
            print("*" * 14 + "aa number out of bounds")
            return
        
        snp_left = (aa_n - 1) * 3
        snp_right = aa_n * 3
        codon = cds_dna[snp_left:snp_right]
        
        print("Observed codon translation:", codon.translate(), ' expected ', aa_wt)
        
        if aa_wt != codon.translate():
            print("*" * 14 + "Observed aa did not match expected")
            return
        
        product_size_full = len(cds_dna)
        product_size_left = len(cds_dna[:snp_right])
        product_size_right = len(cds_dna[snp_left:])
        
        if os.path.exists('outputs/pcr_product_sizes.txt'):
            f = open('outputs/pcr_product_sizes.txt', 'a')
        else:
            f = open('outputs/pcr_product_sizes.txt', 'w')
            f.write("seqid\tfull_cds_length\tleft\tright\t-\t#AA\tmut\tkDa\tseqid\n")

        f.write(f"{gene_name}\t{product_size_full}\t{product_size_left}")
        f.write(f'\t{product_size_right}')
        f.write("\t-\t{}\t{}".format(product_size_full/3 - 1, f"{aa_wt}{aa_n}{aa_mut}"))
        f.write("\t{:0.3}".format(
            Bio.SeqUtils.molecular_weight(''.join([x for x in cds_aa if x.isalpha()]), seq_type='protein')/1000))
        f.write(f'\t{gene_name}\n')
        
        # Amplification primers for whole cds into
        # BamHI cut Nt FHH pLEX uORF.
        pm = primerMaker(blocks=[str(cds_dna)], left=left_overlap, right=right_overlap)
        pm.pick_primers()
        
        with open('fasta/{}.fa'.format(gene_name), 'w') as f:
            f.write(pm.get_seq_as_fasta())
            
        lines = pm.one_per_line()
        
        lines = '\n'.join([x for x in lines.split('\n') if 'stit' in x])
        bc_acc = re.sub('BC0', '', seq_record.id)
        lines = re.sub('_0_', '', lines)
        lines = re.sub('leftstitch', 'L_{}'.format(gene_name), lines)
        lines = re.sub('rightstitch', 'R_{}'.format(gene_name), lines)

        return lines + '\n'
    

order = ''
genbank_files = glob.glob('gb/*.gb')

def parse_mutation(_str, return_no_mutation=False) -> tuple:
    a = re.split('(\d+)', _str)
    if return_no_mutation:
        return a[0], int(a[1]), a[0]
    return a[0], int(a[1]), a[2]


def get_fname(gene_name: str) -> str:
    
    if type(gene_name) != type(''):
        return False
    
    matching_gbs = [x for x in genbank_files if gene_name in str(x)]
    if len(matching_gbs):
        fname = matching_gbs[0]
    else:
        fname = 'gb/{}.gb'.format(df.iloc[n]['URL'])
        
    if not os.path.exists(fname):
        print(f"{gene_name}: No file {fname}")
        return False
    
    return fname

# Read all CDSes from genbank files and write to a single fasta
# in case this is handy later for BLASTing against expected 
# constructs.
all_cds = ''
cds = {}
for n in range(df.shape[0]):
    
    gene_name = df.iloc[n]['Gene']
    fname = get_fname(gene_name)
    
    print(f"{gene_name}->{fname}")
    if fname:
        print("...->")
        for seq_record in SeqIO.parse(fname, "genbank"):
            cds_dna, cds_aa, starts_with = get_cds_from_genbank_record(seq_record)
            all_cds += f">{gene_name}_{seq_record.id}_{fname}\n"
            all_cds += f"{cds_aa}\n"
            cds[gene_name] = cds_aa
            
with open('outputs/all_expected_cds_from_genbank_input_files.fa', 'w') as f:
    f.write(all_cds)
    
print('Exported all CDSes from genbank files.')

# Clear the file outputs/pcr_product_sizes.txt that is used
# for checking PCR lengths and mutations by hand.
if os.path.exists('outputs/pcr_product_sizes.txt'):
    os.system('rm outputs/pcr_product_sizes.txt')

# Get primers for each mutation.
done = set()
for n in range(df.shape[0]):
    print('---=---' * 7)
    
    gene_name = df.iloc[n]['Gene']
    
    if gene_name not in proteins:
        continue
    
    fname = get_fname(gene_name)
    
    if fname in done:
        continue
    done.add(fname)

    print(gene_name, fname)
    
    if fname:
        mut = parse_mutation(
            df.iloc[n]['Mutation'])#, return_no_mutation=1)
        #if gene_name in cds:
        #    midpoint = int(len(cds[gene_name])/2)
        #    mut = cds[gene_name][midpoint], midpoint + 1, cds[gene_name][midpoint]
        #else:
        #    print(f"No CDS {gene_name}")
        
        lines = get_primers(fname, *mut, gene_name=df.iloc[n]['Gene'], remove_stop=remove_stop)
        print(lines)
        if lines:
            order += lines

#Write a text order sheet.
#Well Position	Name	Sequence
#A1	test1	agagagaagagagagag
with open('order.txt', 'w') as f:
    f.write(order)

def well_value(well: str) -> int:
    row = 'ABCDEFGHIJKLMNOPQRST'.index(well[0])
    value = row * 12 + int(well[1:])
    return value

# Write excel order sheet.
rows = []
alphabet = 'ABCDEFGHIJKLMNOPQRST'
letter_index = 0
number_index = 1
for li in order.split('\n'):
    
    # For now, skip the odd man out.
    #if 'RARS2' in li:
    #    continue
    #if li.count('_') < 2:
    #    continue
    if 'RARS2' in li or ('_0L_' not in li and '_1R_' not in li):
        if len(li.split('\t')) < 2:
            continue
            
        gene_name = li.split('\t')[0].split('_')[-1]
        
        if gene_name not in prot_to_well:
            print(f'Skipping {gene_name}')
            continue
        
        well = prot_to_well[gene_name]
        _well_value = well_value(well)
        if li.split('\t')[0][:2] == 'R_':
            well = well[0] + str(int(well[1:]) + 1)
        #    _well_value += 1
        _well_value = well_value(well)
        rows.append({
            'Well Position': well,#alphabet[letter_index]+str(number_index),
            'Name': li.split('\t')[0],
            'Sequence': li.split('\t')[1].rstrip('\n'),
            'Gene name': li.split('\t')[0].split('_')[-1],
            'sorting': _well_value,
        })
        
        if number_index >= 8:
            letter_index += 1
            number_index = 1
        else:
            number_index += 1
        if letter_index > 7:
            letter_index = 0
            number_index = 1
            
order_df = pandas.DataFrame(rows)
order_df.sort_values(by='sorting', inplace=True)
order_df.to_excel('order.xlsx', )

print(order_df)

In [ ]:
import subprocess
"""
Charlie8
./hg38re/Charlie8
stdout:  output//pair1_/Charlie8.txt
bowtie2 -k 1 -p 1 --quiet -x ./hg38re/Charlie8 short_multimap_R1.fastq | grep "repname" -
stdout:  output//pair2_/Charlie8.txt
bowtie2 -k 1 -p 1 --quiet -x ./hg38re/Charlie8 short_multimap_R2.fastq | grep "repname" -
"""
b_opt = "-k 1 -p 1 --quiet"
file1 = '/Users/dp/pma/dataAndScripts/clip/RepEnrich2/output//pair1_/U8.txt'
metagenomepath = './hg38re/U8'
fastqfile_1 = 'short_multimap_R1.fastq'

with open(file1, 'w') as stdout:
    cmd = 'bowtie2 ' + b_opt + ' ' + '-x ' + metagenomepath + ' ' + fastqfile_1 + ' | grep \"repname\" -'
    cmd = ['bowtie2', '-k', '1', '-p', '1', '--quiet', '-x', metagenomepath, fastqfile_1]
    #cmd = 'echo hi'
    print(cmd)
    p = subprocess.check_output(cmd, shell=True)#, stdout=stdout)
#    p.communicate()
    print(p)
stdout.close()
with open(file1) as f:
    print(f.readlines())

In [ ]:
with open(file1) as f:
    print(f.readlines())